# EA solution to the TSP problem

In [22]:
import logging
from itertools import combinations
from itertools import accumulate
from dataclasses import dataclass
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
#from pyvis.network import Network
from matplotlib import pyplot as plt

from tqdm.auto import tqdm
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [23]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
# It return r-length tuples in sorted order with no repeated elements 
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [24]:
def tsp_cost(tsp):
    #assert -> se la condizione torna true tutto ok, altrimenti lancia un errore
    #check che il primo e l'ultimo elemento siano uguali
    assert tsp[0] == tsp[-1]
    #check che tutte le città siano presenti e che non ci siano duplicati
    assert set(tsp) == set(range(len(CITIES))), print(tsp)

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [25]:
TAU = 2 #tau = 2 same selective pressure as linear ranking selection -> I could make this adaptive

@dataclass
class Individual:
    tsp: list
    fitness: float = None
    #distance: float = None

def fitness(ind: Individual):
    if ind.fitness is None:
        ind.tsp.append(ind.tsp[0])
        ind.fitness = -tsp_cost(ind.tsp)
        ind.tsp.pop()
    #fare una prova anche con la distanza quaratica
    return ind.fitness

#parent selection -> tournament selection
def parent_selection(population):
    #verificare che funzioni
    candidates = sorted(np.random.choice(population, TAU), key=fitness, reverse=True)
    return candidates[0]

def insert_mutation(p: Individual):
    newTsp = p.tsp.copy()
    i, j = np.random.choice(len(newTsp), 2, replace=False)
    if i > j: i, j = j, i
    #print(i,j)
    temp = newTsp[j]
    newTsp[i+2:j+1]=newTsp[i+1:j]
    newTsp[i+1]=temp
    #print(newTsp)
    return Individual(newTsp)

#takes one gene from the first parent and one edge from the second parent
def inver_over_xover(p1: Individual, p2: Individual):
    newTsp = p2.tsp.copy()
    i = np.random.choice(len(newTsp)-1)
    #I take two adjacent nodes from the first parent -> I retrieve an edge
    node1, node2 = p1.tsp[i], p1.tsp[i+1]
    #Find the indexes of the nodes in the second parent
    id1 = newTsp.index(node1)
    id2 = newTsp.index(node2)   
    if id1 > id2: 
        id1, id2 = id2, id1 
        node2 = node1
    #recreate the edge in the offspring and reverse what's in between
    newTsp[id1+2:id2+1] = newTsp[id1+1:id2][::-1]
    newTsp[id1+1] = node2 
    
    return Individual(newTsp)

In [26]:
#trials
# v1 = Individual([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# v2 = insert_mutation(v1)
# v2

In [27]:
def greedyFirstSolution(firstCity):
    
    city = firstCity

    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    visited[city] = True
    
    #tsp per come è stato implementato è una lista di interi
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        #ritorna l'indice della città più vicina
        closest = np.argmin(dist[city])
        #printEdge(city, closest)
        visited[closest] = True
        city = closest
        tsp.append(int(city))
        #printEdge(tsp[-1], tsp[0])
    tsp.append(tsp[0])
    return tsp

In [28]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 500
MAX_GENERATIONS = 1000
MUTATION_PROBABILITY = .6

#define population 
#Here I implicitly accept copies of the same first solution
#try: using the list of flags to check if a solution has already been used
population = [Individual(greedyFirstSolution(np.random.randint(len(CITIES)))) for _ in range(POPULATION_SIZE)]
for i in population:
    i.fitness = fitness(i)

for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    #hypermodern approach -> we choose if we want to do mutation or crossover
    for _ in range(OFFSPRING_SIZE):
        if np.random.random() < MUTATION_PROBABILITY:
            #do mutation
            p1 = parent_selection(population)
            o = insert_mutation(p1)
            #print("mutation: ", o.tsp)
        else:
            #do crossover
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = inver_over_xover(p1, p2)
            #print("crossover: ", o.tsp)
        offspring.append(o)
    
    
    for i in offspring:
        #print(i.tsp)
        i.fitness = fitness(i)
    
    population.extend(offspring)
    population = sorted(population, key=fitness, reverse=True)[:POPULATION_SIZE]
    print(f"Generation {g} best cost: {max(population, key=fitness).fitness}")

print(f"Best cost: {max(population, key=fitness).fitness}")


  0%|          | 0/1000 [00:00<?, ?it/s]

Generation 0 best cost: -4432.413117764403
Generation 1 best cost: -4432.413117764403
Generation 2 best cost: -4432.413117764403
Generation 3 best cost: -4297.255090046364
Generation 4 best cost: -4297.255090046364
Generation 5 best cost: -4297.255090046364
Generation 6 best cost: -4297.255090046364
Generation 7 best cost: -4297.255090046364
Generation 8 best cost: -4290.851920578198
Generation 9 best cost: -4290.851920578198
Generation 10 best cost: -4286.183918990257
Generation 11 best cost: -4286.183918990257
Generation 12 best cost: -4286.183918990257
Generation 13 best cost: -4286.183918990257
Generation 14 best cost: -4286.183918990257
Generation 15 best cost: -4286.183918990257
Generation 16 best cost: -4286.183918990257
Generation 17 best cost: -4286.183918990257
Generation 18 best cost: -4286.183918990257
Generation 19 best cost: -4286.183918990257
Generation 20 best cost: -4286.183918990257
Generation 21 best cost: -4286.183918990257
Generation 22 best cost: -4285.12859066929